The nv center paper https://pubs.aip.org/avs/aqs/article/2/2/024701/997281/Introduction-to-quantum-optimal-control-for has an example of a control pulse for just the zeeman term of the Hamiltonian.  I want to replicate this and put it in GRAPE to try to understand how to deal with the B-field being aligned or not aligned with the NV-axes.

In [15]:
import matplotlib.pyplot as plt
import time
import numpy as np
from numpy import pi

from scipy.constants import physical_constants, h
from dataclasses import dataclass

from qutip import *
from qutip.control import *
from qutip.qip.operations import rx
from qutip.qip.operations import ry
from qutip.qip.operations import rz

from qutip.control.grape import plot_grape_control_fields, _overlap
from qutip.control.cy_grape import cy_overlap
from qutip.control.grape import cy_grape_unitary, grape_unitary_adaptive

from scipy.interpolate import interp1d
from qutip.ui.progressbar import TextProgressBar

In [21]:
'''def rz3(phi, N=None, target=0):
    
    rotationx=Qobj([[np.exp(-1j*(phi/2)),0,0],
                            [0,0,0],
                            [0,0,-np.exp(1j*(phi/2))]])
    
    return rotationx


T = 1
times = np.linspace(0, T, 100)

#theta, phi = np.random.rand(2)
theta, phi = [pi/2, pi/4]

# target unitary transformation (random single qubit rotation)
#U = rz(phi) * rx(theta) 
#U

U= rz3(phi) #* rx3(theta) 
U'''

'def rz3(phi, N=None, target=0):\n    \n    rotationx=Qobj([[np.exp(-1j*(phi/2)),0,0],\n                            [0,0,0],\n                            [0,0,-np.exp(1j*(phi/2))]])\n    \n    return rotationx\n\n\nT = 1\ntimes = np.linspace(0, T, 100)\n\n#theta, phi = np.random.rand(2)\ntheta, phi = [pi/2, pi/4]\n\n# target unitary transformation (random single qubit rotation)\n#U = rz(phi) * rx(theta) \n#U\n\nU= rz3(phi) #* rx3(theta) \nU'

The following cells are to write the Hamiltonian using the rotating wave approximation

In [29]:
R = 150
H_ops = [sigmax(), sigmay(), sigmaz()]

#Labels for the graph
H_labels = [r'$u_{x}$',
            r'$u_{y}$',
            r'$u_{z}$',
        ]

In [18]:
#defining constants in hamiltonian
h= 6.582119 * 1e-16 #eV * sec ???????????? what h bar to use
D= 2.87 * 1e9 #given in GHz converted to Hz
E= 5.2 * 1e6 #given in MHz converted to Hz 
delta_parallel= 0.17 #Hz/(V*m^-1)
delta_perpendicular= 1e-3 #Hz/(V*m^-1)
epsilonX= 1.31 * 1e6 #given in MHz converted to Hz
epsilonY= 1.31 * 1e6 #given in MHz converted to Hz
epsilonZ= 1.31 * 1e6 #given in MHz converted to Hz

#squaring matrices
sigmaX3squared=sigmax3*sigmaX3
sigmaY3squared=sigmaY3*sigmaY3
sigmaZ3squared=sigmaZ3*sigmaZ3

In [30]:
H0= h * delta_parallel * epsilonZ * (sigmaZ3squared-(2/3)) - ((h * delta_perpendicular) \
        * ((epsilonX * ((sigmaX3*sigmaY3) + (sigmaY3*sigmaX3))) + (epsilonY * (sigmaX3squared - sigmaY3squared))))

H0

Quantum object: dims = [[3], [3]], shape = (3, 3), type = oper, isherm = True
Qobj data =
[[-2.44306317e-11  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -9.77225268e-11  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -2.44306317e-11]]

In [22]:
#writing out starting state for single qubit

psi0 = basis(2, 0)
print (psi0)

Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
Qobj data =
[[1.]
 [0.]]


In [23]:
#creating transform matrix

phi=(np.pi/2)
U = ry(phi)
U

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = False
Qobj data =
[[ 0.70710678 -0.70710678]
 [ 0.70710678  0.70710678]]

In [24]:
#definiting U0 ??????

u0 = np.array([np.random.rand(len(times)) * 2 * pi * 0.005 for _ in range(len(H_ops))])
u0 = [np.convolve(np.ones(10)/10, u0[idx,:], mode='same') for idx in range(len(H_ops))]

In [27]:
#doing GRAPE!
result = cy_grape_unitary(U, H0, H_ops, R, times, u_start=u0, eps=2*pi/T, phase_sensitive=False,
                          progress_bar=TextProgressBar())

ValueError: dimension mismatch

ok now I'm going to write out the full Hamiltonian for that term

In [13]:
#defining hamiltonian for NV center

#constants

f_fine_structure = 2.87e9
f_nuclear_quadrupole = -5.01e6
f_axial_magnetic_hyperfine = -2.14E6
f_transverse_magnetic_hyperfine = -2.7E6
g_factor_electron = 2.0028
gyromagnetic_constant_nuclear = 1.93297E7 / (2 * np.pi)
uB = physical_constants['Bohr magneton'][0]
uN = physical_constants['nuclear magneton'][0]    
bvector=[0,0,1]



@dataclass
class NVGroundParameters14N:
    nuclear_spin: int = 1
    electron_spin: int = 1
    f_fine_structure: float = 2.87E9
    f_nuclear_quadrupole: float = -5.01E6
    f_axial_magnetic_hyperfine: float = -2.14E6
    f_transverse_magnetic_hyperfine: float = -2.7E6
    g_factor_electron: float = 2.0028
    gyromagnetic_constant_nuclear: float = 1.93297E7 / (2 * np.pi) # Hz / Tesla

        
bvector=[0,0,1]

def nnplus1(n):
    return n * (n + 1)


def twonplus1(n):
    return 2 * n + 1

In [14]:
def get_nv_zeeman_hamiltonian(p:NVGroundParameters14N, bvector):
    """
    Zeeman Hamiltonian for both electronic and nuclear spin.
    :param p: Ground state NV center parameters
    :param bvector: Static magnetic field vector
    :return: The Zeeman Hamiltonian for a static magnetic field
    """
    h_zeeman = uB / h * p.g_factor_electron * tensor(jmat(p.electron_spin, 'x') * bvector[0] +
                                            jmat(p.electron_spin, 'y') * bvector[1] +
                                            jmat(p.electron_spin, 'z') * bvector[2], identity(twonplus1(p.nuclear_spin))) + \
          p.gyromagnetic_constant_nuclear * (tensor(identity(twonplus1(p.electron_spin)),
                                                    jmat(p.nuclear_spin, 'x') * bvector[0] +
                                                    jmat(p.nuclear_spin, 'y') * bvector[1] +
                                                    jmat(p.nuclear_spin, 'z') * bvector[2]))
    return h_zeeman